In [3]:
import pandas as pd
import numpy as np
from prophet import Prophet
from sklearn.metrics import mean_squared_error
import sys
from pathlib import Path

# Zorg dat build_training_set geïmporteerd is
current_dir = Path.cwd()
while current_dir.name != "ENEXIS" and current_dir.parent != current_dir:
    current_dir = current_dir.parent
project_root = current_dir
utils_path = project_root / "src" / "utils"
sys.path.append(str(utils_path))
from build_training_set import build_training_set

target = 'Price'

# Initial training window
base_start = "2025-01-01 00:00:00"
base_end = "2025-03-14 23:00:00"
base_run = "2025-03-15 00:00:00"

rmse_results = []

print("🔍 Testing Prophet Model - RMSE per forecast day")
print("=" * 60)

for i in range(30):
    start = pd.Timestamp(base_start) + pd.Timedelta(days=i)
    end = pd.Timestamp(base_end) + pd.Timedelta(days=i)
    run_date = pd.Timestamp(base_run) + pd.Timedelta(days=i)

    try:
        df = build_training_set(
            train_start=start.strftime("%Y-%m-%d %H:%M:%S"),
            train_end=end.strftime("%Y-%m-%d %H:%M:%S"),
            run_date=run_date.strftime("%Y-%m-%d %H:%M:%S")
        )

        if df is None or df.empty:
            print(f"Day {i+1}: ❌ No training data returned")
            continue

        df['target_datetime'] = pd.to_datetime(df['target_datetime'], utc=True)
        df = df.sort_values('target_datetime')

        run_date_utc = run_date.tz_localize("UTC")

        # Split into training and testing sets
        train_data = df[df['target_datetime'] <= run_date_utc]
        test_data = df[df['target_datetime'] > run_date_utc]

        # Drop any missing data in training
        train_data = train_data.dropna(subset=['target_datetime', target])

        if test_data.empty or train_data.empty:
            print(f"Day {i+1}: ❌ Not enough data for training or testing")
            continue

        # Prepare data for Prophet
        prophet_train = train_data.rename(columns={'target_datetime': 'ds', target: 'y'})[['ds', 'y']]
        prophet_train['ds'] = prophet_train['ds'].dt.tz_localize(None)
        prophet_test = test_data.rename(columns={'target_datetime': 'ds', target: 'y'})[['ds', 'y']]
        prophet_test['ds'] = prophet_test['ds'].dt.tz_localize(None)

        # Train Prophet model
        model = Prophet(daily_seasonality=True, yearly_seasonality=True, weekly_seasonality=True)
        model.fit(prophet_train)

        # Forecast for the test period
        future = prophet_test[['ds']]
        forecast = model.predict(future)
        y_pred = forecast['yhat'].values
        y_test = prophet_test['y'].values

        # Sla de eerste 24 uur over
        if len(y_pred) > 24:
            y_pred = y_pred[24:]
            y_test = y_test[24:]
        else:
            print("Niet genoeg testdata na lag van 24 uur.")
            rmse = np.nan
            rmse_results.append({
                'iteration': i + 1,
                'run_date': run_date.strftime('%Y-%m-%d'),
                'valid_predictions': 0,
                'rmse': rmse
            })
            continue

        if len(y_pred) > 0:
            rmse = np.sqrt(mean_squared_error(y_test, y_pred))
        else:
            rmse = np.nan

        rmse_results.append({
            'iteration': i + 1,
            'run_date': run_date.strftime('%Y-%m-%d'),
            'valid_predictions': len(y_pred),
            'rmse': rmse
        })

        print(f"Day {i+1}: ✅ {len(y_pred)} test rows, Run: {run_date.strftime('%m-%d')}")

    except Exception as e:
        print(f"Day {i+1}: ❌ Error: {e}")

# Create results dataframe
if rmse_results:
    rmse_df = pd.DataFrame(rmse_results)

    print(f"\n📊 OVERALL RMSE - Prophet Model")
    print("=" * 80)
    print(f"Successful runs: {rmse_df['rmse'].notna().sum()}/30")

    print(rmse_df[['iteration', 'run_date', 'valid_predictions', 'rmse']].round(2).to_string(index=False))

    print(f"\n📈 SUMMARY STATISTICS")
    print("-" * 40)
    print(rmse_df['rmse'].describe().round(2))

    print(f"\n📊 AVERAGE OVERALL RMSE")
    print("-" * 40)
    print(f"Mean RMSE: {rmse_df['rmse'].mean():.4f}")
    print(f"Stddev RMSE: {rmse_df['rmse'].std():.4f}")

else:
    print("❌ No runs completed successfully")

2025-05-30 15:54:20,283 - build_training_set - INFO - 📅 Loading additional historical data until 2025-03-15 00:00:00+00:00 for lagging support
2025-05-30 15:54:20,287 - build_training_set - INFO - 🚀 Start build van trainingset
2025-05-30 15:54:20,289 - build_training_set - INFO - 🧠 Actuals van 2025-01-01 00:00:00+00:00 t/m 2025-03-14 23:00:00+00:00 (extended to 2025-03-15 00:00:00+00:00 for lagging)
2025-05-30 15:54:20,292 - build_training_set - INFO - 📅 Forecast van run_date 2025-03-15 00:00:00+00:00, normalized to 2025-03-15 00:00:00+00:00 for DB lookup, target range: 2025-03-15 00:00:00+00:00 → 2025-03-22 00:00:00+00:00
2025-05-30 15:54:20,295 - build_training_set - INFO - 📥 Loading actuals with selected columns only...
2025-05-30 15:54:20,300 - build_training_set - INFO - 📋 Requested columns found: 21/21
2025-05-30 15:54:20,301 - build_training_set - INFO - 📋 Using columns: ['Price', 'target_datetime', 'Load', 'shortwave_radiation', 'temperature_2m', 'direct_normal_irradiance', 'di

🔍 Testing Prophet Model - RMSE per forecast day


2025-05-30 15:54:20,473 - build_training_set - INFO - 🔧 Missing columns in predictions: ['Price', 'Load', 'Flow_NO', 'Flow_GB']
2025-05-30 15:54:20,473 - build_training_set - INFO - 📊 Applying 168-hour lag for missing columns (excluding target variables)...
2025-05-30 15:54:20,473 - build_training_set - INFO -    🎯 Column 'Price' is target variable - filled with NaN (not lagged)
2025-05-30 15:54:20,473 - build_training_set - INFO -    🕐 Lagging column 'Load' by 168 hours
2025-05-30 15:54:20,564 - build_training_set - INFO -    ✅ Added Load: 169/169 values found
2025-05-30 15:54:20,570 - build_training_set - INFO -    🕐 Lagging column 'Flow_NO' by 168 hours
2025-05-30 15:54:20,651 - build_training_set - INFO -    ✅ Added Flow_NO: 169/169 values found
2025-05-30 15:54:20,657 - build_training_set - INFO -    🕐 Lagging column 'Flow_GB' by 168 hours
2025-05-30 15:54:20,814 - build_training_set - INFO -    ✅ Added Flow_GB: 169/169 values found
2025-05-30 15:54:20,814 - build_training_set - I

Day 1: ✅ 144 test rows, Run: 03-15


2025-05-30 15:54:22,394 - build_training_set - INFO -    ✅ Added Load: 169/169 values found
2025-05-30 15:54:22,402 - build_training_set - INFO -    🕐 Lagging column 'Flow_NO' by 168 hours
2025-05-30 15:54:22,533 - build_training_set - INFO -    ✅ Added Flow_NO: 169/169 values found
2025-05-30 15:54:22,533 - build_training_set - INFO -    🕐 Lagging column 'Flow_GB' by 168 hours
2025-05-30 15:54:22,609 - build_training_set - INFO -    ✅ Added Flow_GB: 169/169 values found
2025-05-30 15:54:22,609 - build_training_set - INFO - 🔄 Combining actuals and predictions...
2025-05-30 15:54:22,609 - build_training_set - INFO - 💰 Retrieving actual prices for forecast period...
2025-05-30 15:54:22,617 - build_training_set - INFO - 📊 Found 169 actual prices for forecast period
2025-05-30 15:54:22,674 - build_training_set - INFO - ✅ Filled 169/169 prediction prices with actual values
2025-05-30 15:54:22,683 - build_training_set - INFO - 💰 Price coverage: 169/169 (100.0%)
2025-05-30 15:54:22,689 - buil

...

Day 28: ✅ 144 test rows, Run: 04-11


2025-05-30 15:55:01,325 - build_training_set - INFO -    🕐 Lagging column 'Load' by 168 hours
2025-05-30 15:55:01,458 - build_training_set - INFO -    ✅ Added Load: 169/169 values found
2025-05-30 15:55:01,458 - build_training_set - INFO -    🕐 Lagging column 'Flow_NO' by 168 hours
2025-05-30 15:55:01,599 - build_training_set - INFO -    ✅ Added Flow_NO: 169/169 values found
2025-05-30 15:55:01,607 - build_training_set - INFO -    🕐 Lagging column 'Flow_GB' by 168 hours
2025-05-30 15:55:01,731 - build_training_set - INFO -    ✅ Added Flow_GB: 169/169 values found
2025-05-30 15:55:01,740 - build_training_set - INFO - 🔄 Combining actuals and predictions...
2025-05-30 15:55:01,740 - build_training_set - INFO - 💰 Retrieving actual prices for forecast period...
2025-05-30 15:55:01,749 - build_training_set - INFO - 📊 Found 169 actual prices for forecast period
2025-05-30 15:55:01,807 - build_training_set - INFO - ✅ Filled 169/169 prediction prices with actual values
2025-05-30 15:55:01,815 -

Day 29: ✅ 144 test rows, Run: 04-12


2025-05-30 15:55:02,940 - build_training_set - INFO -    ✅ Added Flow_NO: 169/169 values found
2025-05-30 15:55:02,940 - build_training_set - INFO -    🕐 Lagging column 'Flow_GB' by 168 hours
2025-05-30 15:55:03,075 - build_training_set - INFO -    ✅ Added Flow_GB: 169/169 values found
2025-05-30 15:55:03,084 - build_training_set - INFO - 🔄 Combining actuals and predictions...
2025-05-30 15:55:03,084 - build_training_set - INFO - 💰 Retrieving actual prices for forecast period...
2025-05-30 15:55:03,092 - build_training_set - INFO - 📊 Found 169 actual prices for forecast period
2025-05-30 15:55:03,155 - build_training_set - INFO - ✅ Filled 169/169 prediction prices with actual values
2025-05-30 15:55:03,155 - build_training_set - INFO - 💰 Price coverage: 169/169 (100.0%)
2025-05-30 15:55:03,163 - build_training_set - INFO - ✅ Combined dataset: 1921 rows (1752 actuals + 169 predictions)
2025-05-30 15:55:03,165 - build_training_set - INFO - ✅ No overlap between actuals and predictions
202

Day 30: ✅ 144 test rows, Run: 04-13

📊 OVERALL RMSE - Prophet Model
Successful runs: 30/30
 iteration   run_date  valid_predictions  rmse
         1 2025-03-15                144  0.14
         2 2025-03-16                144  0.15
         3 2025-03-17                144  0.10
         4 2025-03-18                144  0.08
         5 2025-03-19                144  0.04
         6 2025-03-20                144  0.04
         7 2025-03-21                144  0.05
         8 2025-03-22                144  0.05
         9 2025-03-23                144  0.05
        10 2025-03-24                144  0.06
        11 2025-03-25                144  0.12
        12 2025-03-26                144  0.13
        13 2025-03-27                144  0.10
        14 2025-03-28                144  0.06
        15 2025-03-29                144  0.06
        16 2025-03-30                144  0.08
        17 2025-03-31                144  0.24
        18 2025-04-01                144  0.07
        19 2025-

In [4]:
import pandas as pd
import numpy as np
from prophet import Prophet
from sklearn.metrics import mean_squared_error
import sys
from pathlib import Path

# Zorg dat build_training_set geïmporteerd is
current_dir = Path.cwd()
while current_dir.name != "ENEXIS" and current_dir.parent != current_dir:
    current_dir = current_dir.parent
project_root = current_dir
utils_path = project_root / "src" / "utils"
sys.path.append(str(utils_path))
from build_training_set import build_training_set

FEATURES = [
    'Load', 'shortwave_radiation', 'temperature_2m', 
    'direct_normal_irradiance', 'diffuse_radiation', 
    'Flow_NO', 'yearday_cos', 'Flow_GB', 'month', 'is_dst', 
    'yearday_sin', 'wind_speed_10m', 'is_non_working_day', 
    'hour_cos', 'is_weekend', 'cloud_cover', 'weekday_sin', 
    'hour_sin', 'weekday_cos'
]
target = 'Price'

# Initial training window
base_start = "2025-01-01 00:00:00"
base_end = "2025-03-14 23:00:00"
base_run = "2025-03-15 00:00:00"

rmse_results = []

print("🔍 Testing Prophet Model - RMSE per forecast day")
print("=" * 60)

for i in range(30):
    start = pd.Timestamp(base_start) + pd.Timedelta(days=i)
    end = pd.Timestamp(base_end) + pd.Timedelta(days=i)
    run_date = pd.Timestamp(base_run) + pd.Timedelta(days=i)

    try:
        df = build_training_set(
            train_start=start.strftime("%Y-%m-%d %H:%M:%S"),
            train_end=end.strftime("%Y-%m-%d %H:%M:%S"),
            run_date=run_date.strftime("%Y-%m-%d %H:%M:%S")
        )

        if df is None or df.empty:
            print(f"Day {i+1}: ❌ No training data returned")
            continue

        df['target_datetime'] = pd.to_datetime(df['target_datetime'], utc=True)
        df = df.sort_values('target_datetime')

        run_date_utc = run_date.tz_localize("UTC")

        # Split into training and testing sets
        train_data = df[df['target_datetime'] <= run_date_utc]
        test_data = df[df['target_datetime'] > run_date_utc]

        # Drop any missing data in training and test
        train_data = train_data.dropna(subset=['target_datetime', target] + FEATURES)
        test_data = test_data.dropna(subset=['target_datetime', target] + FEATURES)

        if test_data.empty or train_data.empty:
            print(f"Day {i+1}: ❌ Not enough data for training or testing")
            continue

        # Prepare data for Prophet
        prophet_train = train_data.rename(columns={'target_datetime': 'ds', target: 'y'})[['ds', 'y'] + FEATURES]
        prophet_train['ds'] = prophet_train['ds'].dt.tz_localize(None)
        prophet_test = test_data.rename(columns={'target_datetime': 'ds', target: 'y'})[['ds', 'y'] + FEATURES]
        prophet_test['ds'] = prophet_test['ds'].dt.tz_localize(None)

        # Train Prophet model with extra regressors
        model = Prophet(daily_seasonality=True, yearly_seasonality=True, weekly_seasonality=True)
        for reg in FEATURES:
            model.add_regressor(reg)
        model.fit(prophet_train)

        # Forecast for the test period
        future = prophet_test[['ds'] + FEATURES]
        forecast = model.predict(future)
        y_pred = forecast['yhat'].values
        y_test = prophet_test['y'].values

        # Sla de eerste 24 uur over
        if len(y_pred) > 24:
            y_pred = y_pred[24:]
            y_test = y_test[24:]
        else:
            print("Niet genoeg testdata na lag van 24 uur.")
            rmse = np.nan
            rmse_results.append({
                'iteration': i + 1,
                'run_date': run_date.strftime('%Y-%m-%d'),
                'valid_predictions': 0,
                'rmse': rmse
            })
            continue

        if len(y_pred) > 0:
            rmse = np.sqrt(mean_squared_error(y_test, y_pred))
        else:
            rmse = np.nan

        rmse_results.append({
            'iteration': i + 1,
            'run_date': run_date.strftime('%Y-%m-%d'),
            'valid_predictions': len(y_pred),
            'rmse': rmse
        })

        print(f"Day {i+1}: ✅ {len(y_pred)} test rows, Run: {run_date.strftime('%m-%d')}")

    except Exception as e:
        print(f"Day {i+1}: ❌ Error: {e}")

# Create results dataframe
if rmse_results:
    rmse_df = pd.DataFrame(rmse_results)

    print(f"\n📊 OVERALL RMSE - Prophet Model")
    print("=" * 80)
    print(f"Successful runs: {rmse_df['rmse'].notna().sum()}/30")

    print(rmse_df[['iteration', 'run_date', 'valid_predictions', 'rmse']].round(2).to_string(index=False))

    print(f"\n📈 SUMMARY STATISTICS")
    print("-" * 40)
    print(rmse_df['rmse'].describe().round(2))

    print(f"\n📊 AVERAGE OVERALL RMSE")
    print("-" * 40)
    print(f"Mean RMSE: {rmse_df['rmse'].mean():.4f}")
    print(f"Stddev RMSE: {rmse_df['rmse'].std():.4f}")

else:
    print("❌ No runs completed successfully")

2025-05-30 15:58:18,080 - build_training_set - INFO - 📅 Loading additional historical data until 2025-03-15 00:00:00+00:00 for lagging support
2025-05-30 15:58:18,081 - build_training_set - INFO - 🚀 Start build van trainingset
2025-05-30 15:58:18,082 - build_training_set - INFO - 🧠 Actuals van 2025-01-01 00:00:00+00:00 t/m 2025-03-14 23:00:00+00:00 (extended to 2025-03-15 00:00:00+00:00 for lagging)
2025-05-30 15:58:18,083 - build_training_set - INFO - 📅 Forecast van run_date 2025-03-15 00:00:00+00:00, normalized to 2025-03-15 00:00:00+00:00 for DB lookup, target range: 2025-03-15 00:00:00+00:00 → 2025-03-22 00:00:00+00:00
2025-05-30 15:58:18,087 - build_training_set - INFO - 📥 Loading actuals with selected columns only...
2025-05-30 15:58:18,088 - build_training_set - INFO - 📋 Requested columns found: 21/21
2025-05-30 15:58:18,095 - build_training_set - INFO - 📋 Using columns: ['Price', 'target_datetime', 'Load', 'shortwave_radiation', 'temperature_2m', 'direct_normal_irradiance', 'di

🔍 Testing Prophet Model - RMSE per forecast day


2025-05-30 15:58:18,271 - build_training_set - INFO - 🔧 Missing columns in predictions: ['Price', 'Load', 'Flow_NO', 'Flow_GB']
2025-05-30 15:58:18,275 - build_training_set - INFO - 📊 Applying 168-hour lag for missing columns (excluding target variables)...
2025-05-30 15:58:18,275 - build_training_set - INFO -    🎯 Column 'Price' is target variable - filled with NaN (not lagged)
2025-05-30 15:58:18,275 - build_training_set - INFO -    🕐 Lagging column 'Load' by 168 hours
2025-05-30 15:58:18,371 - build_training_set - INFO -    ✅ Added Load: 169/169 values found
2025-05-30 15:58:18,371 - build_training_set - INFO -    🕐 Lagging column 'Flow_NO' by 168 hours
2025-05-30 15:58:18,446 - build_training_set - INFO -    ✅ Added Flow_NO: 169/169 values found
2025-05-30 15:58:18,446 - build_training_set - INFO -    🕐 Lagging column 'Flow_GB' by 168 hours
2025-05-30 15:58:18,513 - build_training_set - INFO -    ✅ Added Flow_GB: 169/169 values found
2025-05-30 15:58:18,513 - build_training_set - I

Day 1: ✅ 144 test rows, Run: 03-15


2025-05-30 15:58:20,154 - build_training_set - INFO -    🕐 Lagging column 'Flow_NO' by 168 hours
2025-05-30 15:58:20,276 - build_training_set - INFO -    ✅ Added Flow_NO: 169/169 values found
2025-05-30 15:58:20,284 - build_training_set - INFO -    🕐 Lagging column 'Flow_GB' by 168 hours
2025-05-30 15:58:20,403 - build_training_set - INFO -    ✅ Added Flow_GB: 169/169 values found
2025-05-30 15:58:20,405 - build_training_set - INFO - 🔄 Combining actuals and predictions...
2025-05-30 15:58:20,405 - build_training_set - INFO - 💰 Retrieving actual prices for forecast period...
2025-05-30 15:58:20,413 - build_training_set - INFO - 📊 Found 169 actual prices for forecast period
2025-05-30 15:58:20,476 - build_training_set - INFO - ✅ Filled 169/169 prediction prices with actual values
2025-05-30 15:58:20,476 - build_training_set - INFO - 💰 Price coverage: 169/169 (100.0%)
2025-05-30 15:58:20,484 - build_training_set - INFO - ✅ Combined dataset: 1921 rows (1752 actuals + 169 predictions)
2025-

...

Day 28: ✅ 144 test rows, Run: 04-11


2025-05-30 15:59:05,148 - build_training_set - INFO -    ✅ Added Load: 169/169 values found
2025-05-30 15:59:05,158 - build_training_set - INFO -    🕐 Lagging column 'Flow_NO' by 168 hours
2025-05-30 15:59:05,291 - build_training_set - INFO -    ✅ Added Flow_NO: 169/169 values found
2025-05-30 15:59:05,293 - build_training_set - INFO -    🕐 Lagging column 'Flow_GB' by 168 hours
2025-05-30 15:59:05,384 - build_training_set - INFO -    ✅ Added Flow_GB: 169/169 values found
2025-05-30 15:59:05,384 - build_training_set - INFO - 🔄 Combining actuals and predictions...
2025-05-30 15:59:05,393 - build_training_set - INFO - 💰 Retrieving actual prices for forecast period...
2025-05-30 15:59:05,394 - build_training_set - INFO - 📊 Found 169 actual prices for forecast period
2025-05-30 15:59:05,466 - build_training_set - INFO - ✅ Filled 169/169 prediction prices with actual values
2025-05-30 15:59:05,466 - build_training_set - INFO - 💰 Price coverage: 169/169 (100.0%)
2025-05-30 15:59:05,466 - buil

Day 29: ✅ 144 test rows, Run: 04-12


2025-05-30 15:59:06,695 - build_training_set - INFO -    ✅ Added Flow_NO: 169/169 values found
2025-05-30 15:59:06,695 - build_training_set - INFO -    🕐 Lagging column 'Flow_GB' by 168 hours
2025-05-30 15:59:06,781 - build_training_set - INFO -    ✅ Added Flow_GB: 169/169 values found
2025-05-30 15:59:06,783 - build_training_set - INFO - 🔄 Combining actuals and predictions...
2025-05-30 15:59:06,783 - build_training_set - INFO - 💰 Retrieving actual prices for forecast period...
2025-05-30 15:59:06,791 - build_training_set - INFO - 📊 Found 169 actual prices for forecast period
2025-05-30 15:59:06,860 - build_training_set - INFO - ✅ Filled 169/169 prediction prices with actual values
2025-05-30 15:59:06,860 - build_training_set - INFO - 💰 Price coverage: 169/169 (100.0%)
2025-05-30 15:59:06,860 - build_training_set - INFO - ✅ Combined dataset: 1921 rows (1752 actuals + 169 predictions)
2025-05-30 15:59:06,869 - build_training_set - INFO - ✅ No overlap between actuals and predictions
202

Day 30: ✅ 144 test rows, Run: 04-13

📊 OVERALL RMSE - Prophet Model
Successful runs: 30/30
 iteration   run_date  valid_predictions  rmse
         1 2025-03-15                144  0.06
         2 2025-03-16                144  0.05
         3 2025-03-17                144  0.03
         4 2025-03-18                144  0.03
         5 2025-03-19                144  0.03
         6 2025-03-20                144  0.03
         7 2025-03-21                144  0.05
         8 2025-03-22                144  0.02
         9 2025-03-23                144  0.04
        10 2025-03-24                144  0.05
        11 2025-03-25                144  0.07
        12 2025-03-26                144  0.05
        13 2025-03-27                144  0.04
        14 2025-03-28                144  0.06
        15 2025-03-29                144  0.09
        16 2025-03-30                144  0.16
        17 2025-03-31                144  0.20
        18 2025-04-01                144  0.12
        19 2025-